In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)

refund = 124690
dividend = 700_000
expense  = 660_000

today = date.today()
today

datetime.date(2021, 12, 30)

### Restart and Run All Cells

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sells.date DESC, name'''
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sells.date DESC, name


In [3]:
sells_df.head().style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,IMH,2021,12,2021,12,12.60,11.00,1.60,"3,000","37,800.00","33,000.00","4,800.00",14.55,"4,643.18",DTD
1,PTTGC,2021,12,2021,4,58.75,66.75,-8.00,"1,000","58,750.00","66,750.00","-8,000.00",-11.99,"-8,277.96",DTD
2,VNT,2021,12,2021,8,38.50,37.50,1.00,"4,500","173,250.00","168,750.00","4,500.00",2.67,"3,742.51",DTD
3,KCE,2021,12,2021,12,91.25,87.00,4.25,"1,000","91,250.00","87,000.00","4,250.00",4.89,"3,855.19",DTD
4,MCS,2021,12,2016,8,14.70,16.70,-2.00,"10,000","147,000.00","167,000.00","-20,000.00",-11.98,"-20,695.48",HD


In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name 
2021       1           2019      8          PTTEP   -38,446.55
                       2020      4          KKP      26,592.67
                                 8          SENA     25,560.10
                                 12         AMATA     7,352.16
                                            COM7     30,238.29
                                                       ...    
           12          2021      11         POPF     -2,513.85
                                            RCL      10,917.59
                                            SIS      19,706.79
                                 12         IMH       4,643.18
                                            KCE       3,855.19
Name: net, Length: 178, dtype: float64

In [5]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/ytd-profit.csv',
 '\\Users\\User\\iCloudDrive\\ytd-profit.csv',
 '\\Users\\User\\Dropbox\\ytd-profit.csv')

In [6]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(box_file)

In [7]:
sells_df.groupby(['sell_month']).net.sum()

sell_month
1      51,296.67
2     202,547.56
3     -19,735.49
4      65,267.16
5      79,180.57
6      14,606.32
7    -102,107.17
8      69,193.08
9       1,544.44
10    -10,446.93
11    119,130.67
12     65,786.40
Name: net, dtype: float64

In [8]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(536263.2799999999, 700000, 1236263.2799999998)

In [9]:
profit = sells_df.net.sum()
net = profit + dividend - expense
expense,dividend,profit, net

(660000, 700000, 536263.2799999999, 576263.2799999998)

### Bottom line of up to date transactions

In [10]:
total = refund + net
total

700953.2799999998